In [ ]:
%load_ext autoreload
%autoreload 2
import time
import tensorflow as tf
import p8_util

In [ ]:
OUTPUT_DIR = './tmp/baseline'
datadir = './data'
is_tensorboard = True

In [ ]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

In [ ]:
import keras

LEARNING_RATE = 1.e-2
nn_type = 'CNN'
nb_class = nClasses
my_feature_columns, loss_reduction, tf_head = p8_util.get_tf_head("images",tuple_dimension, nClasses)
feature_columns = my_feature_columns

optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE)
#optimizer = tf.train.AdagradOptimizer(learning_rate=LEARNING_RATE)
#optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
layer_size = 50
num_layers = 1
learn_mixture_weights = False
dropout = 0.5
seed=p8_util.RANDOM_SEED
is_cnn_batch_norm = True
cnn_layer_config={'feature_map_size':[128]}
initialiser_name = 'xavier'
oNNAdaNetBuilder = p8_util._NNAdaNetBuilder(nn_type, nb_class,feature_columns, optimizer, layer_size, num_layers,
               learn_mixture_weights, dropout, seed, is_cnn_batch_norm,cnn_layer_config, initialiser_name)

In [ ]:
# Parameters that will be provided to model_fn
BATCH_SIZE = 49  #@param {type:"integer"}
MAX_STEPS = 100
params = {'learning_rate':1.e-3, 'net_builder':oNNAdaNetBuilder}
NN_TYPE = 'CNN'
my_config = p8_util.make_config(NN_TYPE,output_dir=OUTPUT_DIR)  

classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=my_config)
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train\
                          , tuple_dimension=tuple_dimension\
                          , training=False\
                          , batch_size=BATCH_SIZE),
        max_steps=MAX_STEPS),
    
    eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("test", x_test, y_test\
                           , tuple_dimension=tuple_dimension\
                           , training=False, batch_size=BATCH_SIZE),
        steps=None))
end_time = time.time()

In [ ]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))


In [ ]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [ ]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

In [ ]:
help(tf.estimator.EstimatorSpec)